In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/9/cronograma/Premier-League-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text(strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

# Exibe a tabela final com os links
print("\nTabela com links:")


Tabela com links:


C:\Users\eduar\AppData\Local\Temp\ipykernel_6500\2417986439.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [2]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sex,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,73.297,Old Trafford,Robert Jones,Relatório da Partida,,None
1,1,sáb,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,30.014,Portman Road Stadium,Tim Robinson,Relatório da Partida,,/pt/partidas/cc5b4244/Manchester-United-Fulham...
2,1,sáb,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,52.196,St James' Park,Craig Pawson,Relatório da Partida,,/pt/partidas/a1d0d529/Ipswich-Town-Liverpool-2...
3,1,sáb,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,29.763,The City Ground,Michael Oliver,Relatório da Partida,,/pt/partidas/34557647/Newcastle-United-Southam...
4,1,sáb,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,39.217,Goodison Park,Simon Hooper,Relatório da Partida,,/pt/partidas/4efc72e4/Nottingham-Forest-Bourne...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,,,,,,,,,,,,,,,None
373,,,,,,,,,,,,,,,None
384,,,,,,,,,,,,,,,None
395,,,,,,,,,,,,,,,None


In [3]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# url = "https://fbref.com/pt/comps/9/cronograma/Premier-League-Resultados-e-Calendarios"
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
# }
# response = requests.get(url, headers=headers)

# if response.status_code == 200:
#     soup = BeautifulSoup(response.content, "html.parser")
    
#     # Verifica a estrutura
#     table_container = soup.find('div', {'class': 'table_container'})
#     if table_container:
#         rows = table_container.find_all('tr')
#         tabela = extrair_tabela(rows)
#         print(tabela)
#     else:
#         print("Tabela não encontrada. Verifique a estrutura HTML.")
# else:
#     print(f"Erro ao acessar a página: {response.status_code}")


In [4]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [5]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [6]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sex,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,73.297,Old Trafford,Robert Jones,Relatório da Partida,,/pt/partidas/cc5b4244/Manchester-United-Fulham...
1,1,sáb,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,30.014,Portman Road Stadium,Tim Robinson,Relatório da Partida,,/pt/partidas/a1d0d529/Ipswich-Town-Liverpool-2...
2,1,sáb,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,52.196,St James' Park,Craig Pawson,Relatório da Partida,,/pt/partidas/34557647/Newcastle-United-Southam...
3,1,sáb,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,29.763,The City Ground,Michael Oliver,Relatório da Partida,,/pt/partidas/4efc72e4/Nottingham-Forest-Bourne...
4,1,sáb,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,39.217,Goodison Park,Simon Hooper,Relatório da Partida,,/pt/partidas/71618ace/Everton-Brighton-and-Hov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,14,qua,2024-12-04,19:30,Southampton,1.6,1–5,5.2,Chelsea,31.193,St Mary's Stadium,Tony Harrington,Relatório da Partida,,/pt/partidas/e4480630/Southampton-Chelsea-2024...
149,14,qua,2024-12-04,20:15,Arsenal,2.1,2–0,0.2,Manchester Utd,60.256,Emirates Stadium,Samuel Barrott,Relatório da Partida,,/pt/partidas/9938aa27/Arsenal-Manchester-Unite...
150,14,qua,2024-12-04,20:15,Aston Villa,2.4,3–1,0.9,Brentford,,Villa Park,Lewis Smith,Relatório da Partida,,/pt/partidas/c24a734b/Aston-Villa-Brentford-20...
151,14,qui,2024-12-05,19:30,Fulham,1.1,3–1,1.4,Brighton,26.368,Craven Cottage,Peter Bankes,Relatório da Partida,,/pt/partidas/32aaf579/Fulham-Brighton-and-Hove...


In [7]:
tabela.to_csv(r'C:\Users\eduar\OneDrive\Desktop\DEV\Football\PremierLeague\links.csv', index= False)